In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
with open('data1.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']

In [3]:
yolo_vechleDetection = cv2.dnn.readNetFromONNX('./vechleDetection/weights/best.onnx')
yolo_vechleDetection.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_vechleDetection.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
yolo_numberPlate = cv2.dnn.readNetFromONNX('./numberPlate/weights/best.onnx')
yolo_numberPlate.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo_numberPlate.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [5]:
def rescaleFrame(frame,scale=0.75):
    
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width,height)
    return cv2.resize(frame,dimensions,interpolation=cv2.INTER_AREA)

In [6]:
img = cv2.imread('./Cars252.png')
cv2.imshow('orginal',img)
cv2.waitKey(0)
image = rescaleFrame(img)
# image = img.copy()
row, col, d = image.shape

In [7]:
# get the YOLO prediction from the the image
#  convert image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image
# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo_vechleDetection.setInput(blob)
preds = yolo_vechleDetection.forward() # detection or prediction from YOLO

In [8]:
print(preds.shape)

(1, 25200, 9)


In [9]:
# Non Maximum Supression
# : filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# widht and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.4:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probabilty occur
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [10]:
index

array([1], dtype=int32)

In [11]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]
    classes_id = 0
    class_name = labels[classes_id]

In [12]:
x,y,w,h,class_name

(12, 20, 332, 233, 'car')

In [13]:
VechleImage = image[y:y+h, x:x+w]
image = VechleImage.copy()
row, col, d = image.shape

In [14]:
# get the YOLO prediction from the the image
#  convert image into square image (array)
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col] = image
# step-2: get prediction from square array
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo_numberPlate.setInput(blob)
preds = yolo_numberPlate.forward() # detection or prediction from YOLO

In [15]:
print(preds.shape)

(1, 25200, 6)


In [16]:
# Non Maximum Supression
#  filter detection based on confidence (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# widht and height of the image (input_image)
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] # confidence of detection an object
    if confidence > 0.4:
        class_score = row[5:].max() # maximum probability from 20 objects
        class_id = row[5:].argmax() # get the index position at which max probabilty occur
        
        if class_score > 0.25:
            cx, cy, w, h = row[0:4]
            # construct bounding from four values
            # left, top, width and height
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            
            box = np.array([left,top,width,height])
            
            # append values into the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)
            
# clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()

In [17]:
# Draw the Bounding
for ind in index:
    # extract bounding box
    x,y,w,h = boxes_np[ind]

In [18]:
numberPlateImage = image[y:y+h, x:x+w]


In [19]:
# cv2.imshow('yolo_prediction',numberPlateImage)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [20]:
import pytesseract # this is tesseract module 
#import matplotlib.pyplot as plt  
import glob 
import os
import string
import random
import datetime
import numpy as np
import pandas as pd

In [21]:
path_to_tesseract = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
pytesseract.tesseract_cmd = path_to_tesseract 
res = ''.join(random.choices(string.ascii_uppercase +string.digits, k=6))
vechlePhoto_address= res +'VP.png'
numberPlate_Photo_address= res +'NP.png'

In [22]:
cv2.imwrite(vechlePhoto_address,image)
cv2.imwrite(numberPlate_Photo_address,numberPlateImage)

True

In [23]:
def rescaleFrame(frame,scale=1.2):
    
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width,height)
    return cv2.resize(frame,dimensions,interpolation=cv2.INTER_CUBIC)
numberPlateImage=rescaleFrame(numberPlateImage)
resize_test_license_plate = cv2.resize( 
    numberPlateImage, None, fx = 2, fy = 2,  
    interpolation = cv2.INTER_CUBIC)
cv2.imwrite(numberPlate_Photo_address,numberPlateImage)

True

In [24]:
grayscale_resize_test_license_plate = cv2.cvtColor( 
    resize_test_license_plate, cv2.COLOR_BGR2GRAY) 

In [25]:
gaussian_blur_license_plate = cv2.GaussianBlur( 
    grayscale_resize_test_license_plate, (5, 5), 0) 

In [26]:
predicted_result = pytesseract.image_to_string(gaussian_blur_license_plate, lang ='eng', 
config ='--oem 3 -l eng --psm 6 -c tessedit_char_whitelist = ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789') 
filter_predicted_result = "".join(predicted_result.split()).replace(":", "").replace("-", "").replace("!","").replace("`","").replace(",","").replace('"',"").replace("[","").replace("]","").replace("|","").replace("'","").replace("_","").replace("s","").replace("a","").replace("b","").replace("c","").replace("d","").replace("e","").replace("f","").replace("g","").replace("h","").replace("i","").replace("j","").replace("k","").replace("l","").replace("m","").replace("n","").replace("o","").replace("p","").replace("q","").replace("r","").replace("s","").replace("t","").replace("u","").replace("v","").replace("w","").replace("x","").replace("y","").replace("z","")
print(filter_predicted_result)

DAN54PJ


In [27]:
current_time = datetime.datetime.now()
date = str(current_time.day)+'/'+str(current_time.month)+"/"+str(current_time.year)
time = str(current_time.hour)+':'+str(current_time.minute)+":"+str(current_time.second)

In [28]:
df= pd.read_csv('GateEntry.csv')

In [29]:
df.loc[len(df.index)] = [class_name,filter_predicted_result, date,time,vechlePhoto_address,numberPlate_Photo_address] 

In [30]:
 #df= df.drop(13)
df.to_csv('GateEntry.csv',index=False)
df

,VechleType,NumberPlate,Date,Time,vechlePhoto,NumberPlatePhoto
0,car,CHIOOSE,9/7/2024,11:55:29,MZSRN2VP.png,MZSRN2NP.png
1,car,IMK3532,9/7/2024,11:58:5,LTJAXLVP.png,LTJAXLNP.png
2,car,CHIOOSE,9/7/2024,15:20:33,VRP0ZAVP.png,VRP0ZANP.png
3,car,IEL194y,9/7/2024,15:27:2,O43EODVP.png,O43EODNP.png
4,car,Q5836,9/7/2024,15:31:23,JE41X6VP.png,JE41X6NP.png
5,car,Q5836,9/7/2024,16:5:12,0JVPEKVP.png,0JVPEKNP.png
6,car,IEL194,9/7/2024,16:7:45,IAPSS9VP.png,IAPSS9NP.png
7,car,IEL194,14/7/2024,11:36:49,6VIIXTVP.png,6VIIXTNP.png
8,car,IEL194,14/7/2024,11:58:57,AQOR6OVP.png,AQOR6ONP.png
9,car,CHIOOSE,14/7/2024,12:0:18,6MQNZJVP.png,6MQNZJNP.png
